# Goal

1. Our goal is to build a basic soybean yield model to predict the coming year's yield of that crop.  We will use features such as temperature (avg) and precipitation, US state, NDVI (measure of quality), podcount and test a shift operator to test the hypothesis that technolical change in the last few years has significantly, on linearly, increased yields

1. Yearly yields:
    
    <font size="3"> $$ Yields_{y} \sim PodCount_{m} + State_{y} + Temp_{m} + Precip_{m} + Quality_{m} + Droughts_{m} + Shift + error_{y} $$ </font>
    

# Long term Goals:
1. Automate process of modeling yield forecasts.
2. Incorporate global data.
3. Provide a range of forecasts.

# Immediate Goals:
1. Use US data to forecast soybean yields.
2. Automate downloading of data and model scoring.
3. Provide figures.
4. Test and compare several forecasting methods.

# <u>Features</u>

## Yields

### Hypothsis: Historical yields are a good indicator of future yields.

1. Plots clearly show a significant trend.
2. Nonlinear (quadratic) term picks up rate of increase.

## Pod Count

### Hypothesis: A good pod count indicates a good final yield.

1. We aren't interested in the final pod count, rather the intermediate counts as a way to predict the final yields (which is highly related to the count).
2. So we need the pod counts for several intermediate months.
3. Available on a monthly basis, starting late in the year (Sep?).



## Weather Features

1. Bad weather in particular months will damage the soybean crops.  
2. Look at min, max and precipitation.
3. Try to identify patterns in extreme events.
4. Polynomials and interactions.

## Quality of Crop
1. This is another feature examining the quality of a crop at a particular moment.
2. Available on a semi-weekly basis. 
3. Not many years available and rather spotty.  John suggested that the quality of these measures isn't good, are there other measures?



## Drought Measures
1. PDSI
2. PHDI
3. PM
4. PZ


## Things to try

1. Ran initial models to get a feel for the data.
2. Added drought data.
3. Try all data, use 

# <u>Models</u>

1. Notebooks to download each feature and put the data into a "model ready" state.
2. A notebook to combine features (joinFeatures.ipynb).
3. A notebook to perform model selection, e.g. ridge and lasso regressions (featureSelection.iypnb).
4. This notebook, which loops over various years and states, performs regressions, and summarize and compares outputs.
5. A helper_Functions.ipynb to hold functions that will otherwise obscure more relevant code.  

### Imports and general settings

In [1]:
import ipynb
from ipynb.fs.full.joinFeatures import mergedData
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
from ipynb.fs.full.helper_Functions import descriptions1

### Import merged data

In [2]:
allData1 = mergedData()

model_allStates_pmdi1.csv
model_allStates_minTemp1.csv
model_allStates_precip.csv
model_allStates_quality.csv
model_allStates_zndx1.csv
model_allStates_pdsi1.csv
model_allStates_podcount.csv
model_allStates_yields.csv
model_allStates_maxTemp1.csv


### Which states and years to train and test

In [263]:
testYear = 2015
startYear = 1800
finalYear = 2021

statesofInterest = ['IOWA', 'INDIANA', 'ILLINOIS', 'KANSAS', 'MINNESOTA', 'MISSOURI', 'NEBRASKA', 'NORTH DAKOTA', 'SOUTH DAKOTA', 'OHIO']
statesofInterest = ['IOWA']

### Start loops over model

In [264]:
myGuesses = []
verbose1 = False
for index1, state1 in enumerate(statesofInterest):
    
    if verbose1 == True:
        print("index: ", index1)
        print("state: ", state1)
    
    stData = allData1[allData1["State"] == state1]
    trainData = stData[stData["Year"] < testYear]
    testData = stData[stData["Year"]  >= testYear]
    
    if verbose1 == True:
         print("train: ", trainData.shape)
         print("train: ", testData.shape)    
    
    ########## Real ##########
    myGuesses.append(pd.DataFrame({"Observed": testData["Yield"]}))
    
    ########## Year Only ##########
    model = smf.ols('Yield ~ Year', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample()
    
    myGuesses.append(pd.DataFrame({"year_only" : predictions}))
    
    if verbose1 == True:
        descriptions1("year_only", model, predictions)
    
    
    ########## Year + Year² ##########
    model = smf.ols('Yield ~ Year + I(Year**2)', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"year__sq" : predictions}))
    
    
    if verbose1 == True:
        descriptions1("year_sq", model, predictions)
        
    ########## Max Temps ##########
    model = smf.ols('Yield ~ Year + I(Year**2) + Maxtemp_Jun + Maxtemp_Jul + Maxtemp_Aug', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"maxTemps" : predictions}))
    
    if verbose1 == True:
        descriptions1("maxTemp", model, predictions)       
        
        
    ########## Precip ##########
    model = smf.ols('Yield ~ Year + I(Year**2) + Precip_Jun + Precip_Jul + Precip_Aug', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"precip" : predictions}))
    
    if verbose1 == True:
        descriptions1("Precip", model, predictions)  
        
        
    ########## Precip + maxTemp ##########
    model = smf.ols('Yield ~ Year + I(Year**2) + Precip_Jul + Precip_Aug + Maxtemp_Jul + Maxtemp_Aug', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"precip_temp" : predictions}))
    
    if verbose1 == True:
        descriptions1("Precip_Temp", model, predictions)    
        
        
    ########## Precip + maxTemp + comb ##########
    model = smf.ols('Yield ~ Year + I(Year**2) + Precip_Jul + Precip_Aug + Maxtemp_Jul + Maxtemp_Aug + I(Precip_Aug**2) + I(Precip_Jul*Precip_Aug)', data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"precip_temp_comb" : predictions}))
    
    if verbose1 == True:
        descriptions1("Precip_Temp_Comb", model, predictions)            
                       
    ########## Lasso ##############################        
    model = smf.ols('Yield ~ Year  + I(Year**2) + I(Maxtemp_Aug**2) + I(Precip_Jun**3) + I(Precip_Jun*Precip_Jul**2) + I(Precip_Jun*Maxtemp_Jun**2) + \
    I(Precip_Jul**3) + I(Precip_Jul**2 * Precip_Aug) + I(Precip_Jul*Precip_Aug**2) + I(Precip_Jul*Maxtemp_Jun**2)+I(Precip_Aug*Maxtemp_Aug**2)' , data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"Lasso" : predictions}))
    
    if verbose1 == True:
        descriptions1("Lasso", model, predictions)  
                
        
    ########## Lasso2 ##############################        
    model = smf.ols('Yield ~ Year + I(Year**2) + I(Year*Pmdi1_Aug) + I(Year*Mintemp_Aug) + \
                     I(Year*Precip_May) + I(Year*Precip_Aug) + I(Year*Maxtemp_Jun) + I(Year*Maxtemp_Aug)' , data=trainData).fit()
    predictions =  model.predict(testData) # predict out of sample
    
    myGuesses.append(pd.DataFrame({"Lasso2" : predictions}))
    
    if verbose1 == False:
        descriptions1("Lasso2", model, predictions)     
    
    ########## ARIMA ##########    
    # ARIMA
    series = trainData["Yield"] 
    order1 = sm.tsa.SARIMAX(series, order=(2, 1, 0), trend='t')
    model = order1.fit()

    predictions = model.forecast(finalYear - testYear)
    df1 = pd.DataFrame({"arima" : predictions})
    df1["Year"] = list(range(testYear, 2021))
    df1.set_index("Year", drop = True, inplace=True)
    
    myGuesses.append(df1)
    
    if verbose1 == True:
        descriptions1("ARIMA", model, predictions)    
    
df1 = pd.concat(myGuesses, axis=1)
df1.head()

def mse1(obs1, col1):
    return(np.sum(np.square(obs1 - col1)))

def mae1(obs1, col1):
    return(np.sum(np.abs(obs1 - col1)))

ans1 = []
for i in df1.columns:
    ans1.append(mse1(df1["Observed"], df1[i]))
df1.loc["Mse"] = ans1
    
ans1 = []
for i in df1.columns:
    ans1.append(mae1(df1["Observed"], df1[i]))   
df1.loc["Mae"] = ans1

df1

Lasso2
                            OLS Regression Results                            
Dep. Variable:                  Yield   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     170.0
Date:                Sun, 20 Sep 2020   Prob (F-statistic):           9.83e-48
Time:                        17:33:03   Log-Likelihood:                -223.73
No. Observations:                  91   AIC:                             465.5
Df Residuals:                      82   BIC:                             488.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              

,Observed,year_only,year__sq,maxTemps,precip,precip_temp,precip_temp_comb,Lasso,Lasso2,arima
Year,,,,,,,,,,
2015,56.5,51.093040,51.999428,53.210228,52.772297,53.426230,53.954169,55.008577,53.029229,48.116066
2016,60.0,51.526143,52.491643,53.149046,54.103430,53.164553,53.212223,56.260870,54.781691,49.508357
2017,57.0,51.959245,52.985128,54.923501,52.887259,54.137113,54.627755,54.696992,53.738749,50.589282
2018,56.0,52.392348,53.479886,53.770756,54.468227,54.261746,55.730000,57.785580,53.943686,50.536814
2019,55.0,52.825450,53.975914,54.939637,53.738420,54.551643,55.203867,55.246913,53.664277,51.384541
2020,58.0,53.258552,54.473213,54.023178,52.715203,52.491433,50.937494,52.796876,52.407877,51.973192
Mse,0.0,166.675832,112.587727,82.858304,97.446937,97.934380,108.176174,51.831041,87.197181,300.702589
Mae,0.0,196.121054,135.682516,101.341958,119.262101,118.401662,127.418399,66.600218,108.131671,341.094337


In [260]:
Precip_Jul Precip_Sep Maxtemp_Sep

SyntaxError: invalid syntax (<ipython-input-260-c73fdc3ef10d>, line 1)

In [66]:
modDfObj = dfObj.apply(lambda x: np.square(x) if x.name in ['x', 'y'] else x)

Index(['Observed', 'year_only', 'year__sq', 'maxTemps', 'precip',
       'precip_temp', 'precip_temp_comb', 'arima'],
      dtype='object')

In [4]:
trainData.head()

NameError: name 'trainData' is not defined

In [ ]:
myGuesses = []
for item, i in enumerate(statesofInterest):
    
    print("item: ", item)
    print("i: ", i)

    ##################################################
    # yield 
    yld1 = pd.read_csv("data_model_ready/model_allStates_yields.csv")
    yld1 = yld1[yld1["Year"] > startYear]
    yld1.set_index('Year', inplace=True, drop=False)
    iowa1 = yld1[yld1["State"] == i]
    real1 = iowa1.tail(5)["Yield"]
    iowa1["Yield"].plot(figsize=(11, 9))
    
    test1 = iowa1[iowa1["Year"] < testYear]
    real_pred = iowa1[iowa1["Year"] >= testYear].Yield

    ##################################################
    # trend only 
    results = smf.ols('Yield ~ Year', data=test1).fit()
    ypred = results.predict(test1["Year"])
   
    Xnew = pd.DataFrame({"Year": list(range(testYear, 2021))})
    Xnewl = list(range(testYear, 2021))
    Xnewc = sm.add_constant(Xnew)
    ynewpred =  results.predict(Xnewc) # predict out of sample
    trend_pred =  results.predict(Xnewc) # predict out of sample

    ##################################################
    # trend plus trend squared
    results2 = smf.ols('Yield ~ Year + I(Year**2)', data=test1).fit()
    
    Xnew = pd.DataFrame({"Year": list(range(testYear, 2021))})
    Xnewc = sm.add_constant(Xnew)
    ynewpred2 =  results2.predict(Xnewc) # predict out of sample
    trend_sqr_pred = results2.predict(Xnewc) # predict out of sample

    ##################################################    
    # ARIMA
    series = test1["Yield"] 
    mod = sm.tsa.SARIMAX(series, order=(2, 1, 2), trend='t')
    # Estimate the parameters
    res = mod.fit()

    arima_pred = res.forecast(2021 - testYear)

    ##################################################
    ## max temp  model
    tmax1 = pd.read_csv("data_model_ready/model_allStates_maxTemp1.csv")
    tmax1.set_index('Year', inplace=True, drop=False)
    tmax1 = tmax1[tmax1["State"] == i]
    tmax1 = tmax1[["Maxtemp_Aug", "Maxtemp_Jul", "Year"]]
    iowa2 = iowa1[["Yield"]]
    iowa3 = iowa2.join(tmax1)

    test1 = iowa3[iowa3["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Maxtemp_Jul + Maxtemp_Aug', data=test1).fit()
   
    train1 = iowa3[iowa3["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    maxTemp_pred =  results.predict(train1) # predict out of sample

    tmin1 = pd.read_csv("data_model_ready/model_allStates_minTemp1.csv")
    tmin1.set_index('Year', inplace=True, drop=False)
    tmin1 = tmin1[tmin1["State"] == i]
    tmin1 = tmin1[["Mintemp_Jul", "Mintemp_Aug", "Year"]]
    tmin1.drop(["Year"], axis=1, inplace=True)
    iowa4 = iowa3.join(tmin1)
    iowa4["AvgTemp_Aug"] = iowa4[['Maxtemp_Aug', 'Mintemp_Aug']].mean(axis=1)
    #iowa4.loc[2020, "AvgTemp_Aug"] = stateAugTemp[item]

    ##################################################
    ## max temp and precip model
    precip1 = pd.read_csv("data_model_ready/model_allStates_precip.csv")
    precip1.set_index('Year', inplace=True, drop=True)
    precip1 = precip1[precip1["State"] == i]
    numRows = precip1.shape[0]
    #precip1.loc[2020,'Precip_Aug'] = statePrecip[item]
    precip1["Year"] = precip1.index
    
    precip1 = precip1[["Precip_Aug", "Precip_Jul", "Precip_Sep"]]
    precip1.replace(-9.99, np.nan, inplace=True)
    iowa5 = iowa4.join(precip1)
    
    test1 = iowa5[iowa5["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Maxtemp_Jul + Maxtemp_Aug + Precip_Jul + I(Precip_Jul**2) + Precip_Aug + I(Precip_Aug**2) + I(Maxtemp_Jul*Maxtemp_Aug)', data=test1).fit()
    
    train1 = iowa5[iowa5["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    ###
    maxT_Precip_pred =  results.predict(train1) # predict out of sample
    
    
    ##################################################
    ## drought data
    pmdi1 = pd.read_csv("data_model_ready/model_allStates_pmdi1.csv")
    pmdi1.set_index('Year', inplace=True, drop=True)
    pmdi1 = pmdi1[pmdi1["State"] == i]
    numRows = pmdi1.shape[0]
    #precip1.loc[2020,'Precip_Aug'] = statePrecip[item]
    pmdi1["Year"] = pmdi1.index
    
    pmdi1 = pmdi1[["Pmdi1_Jun", "Pmdi1_Aug", "Pmdi1_Jul", "Pmdi1_Sep"]]
    pmdi1.replace(-99.99, np.nan, inplace=True)
    iowa6 = iowa5.join(pmdi1)
    
    test1 = iowa6[iowa6["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Pmdi1_Jul + I(Pmdi1_Jul**2) + Pmdi1_Aug + I(Pmdi1_Aug**2) + Maxtemp_Jul + I(Maxtemp_Jul**2) + Maxtemp_Aug + I(Maxtemp_Aug**2) + I(Maxtemp_Jul*Maxtemp_Aug)', data=test1).fit()
    
    train1 = iowa6[iowa6["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    ###
    drought_pred =  results.predict(train1) # predict out of sample
    
    
    ##################################################
    ## drought data 2
    zndx1 = pd.read_csv("data_model_ready/model_allStates_zndx1.csv")
    zndx1.set_index('Year', inplace=True, drop=True)
    zndx1 = zndx1[zndx1["State"] == i]
    numRows = zndx1.shape[0]
    #precip1.loc[2020,'Precip_Aug'] = statePrecip[item]
    zndx1["Year"] = zndx1.index
    
    zndx1 = zndx1[["Zndx1_Jun", "Zndx1_Aug", "Zndx1_Jul", "Zndx1_Sep"]]
    zndx1.replace(-99.99, np.nan, inplace=True)
    iowa7 = iowa6.join(zndx1)
    
    test1 = iowa7[iowa7["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Zndx1_Jul + I(Zndx1_Jul**2) + Zndx1_Aug + I(Zndx1_Aug**2) + Maxtemp_Jul + I(Maxtemp_Jul**2) + Maxtemp_Aug + I(Maxtemp_Aug**2) + I(Maxtemp_Jul*Maxtemp_Aug)', data=test1).fit()
    
    print(results.summary())
        
    train1 = iowa7[iowa7["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    ###
    drought_pred_zndx1 =  results.predict(train1) # predict out of sample

    
    ##################################################
    ## podcount model
    podcount = pd.read_csv("data_model_ready/model_allStates_podcount.csv")
    podcount.set_index('Year', inplace=True, drop=False)
    podcount = podcount[podcount["State"] == i]

    podcount = podcount[["Sep_pod_forecast", "Nov_pod_forecast"]]
    iowa8 = iowa7.join(podcount)

    
    test1 = iowa8[iowa8["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Maxtemp_Aug + Precip_Aug + I(Precip_Aug**2) + Nov_pod_forecast', data=test1).fit()

    train1 = iowa8[iowa8["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    pod_pred =  results.predict(train1) # predict out of sample

    
    ##################################################
    ## Qualtiy model
    quality = pd.read_csv("data_model_ready/model_allStates_quality.csv")
    quality.set_index('Year', inplace=True, drop=False)
    quality = quality[quality["State"] == i]
  
    quality = quality[['WEEK #35PCT EXCELLENT']]
    quality.rename(columns={'WEEK #35PCT EXCELLENT': 'WEEK_35PCT_EXCELLENT'}, inplace=True)
    iowa9 = iowa8.join(quality)
    #####
    iowa9.to_csv("tmp_data/modelData_" + i + ".csv")
    #####
    
    test1 = iowa9[iowa9["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Maxtemp_Aug + Mintemp_Aug + Precip_Aug + I(Precip_Aug**2) + WEEK_35PCT_EXCELLENT', data=test1).fit()
    
    train1 = iowa9[iowa9["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    quality_pred =  results.predict(train1) # predict out of sample

    ##################################################
    ## Mixed model
    test1 = iowa9[iowa9["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2)  + I(Year**3) + Zndx1_Jul + I(Zndx1_Jul**2) + Zndx1_Aug + I(Zndx1_Aug**2) + Maxtemp_Jul + I(Maxtemp_Jul**2) + Maxtemp_Aug + I(Maxtemp_Aug**2) + I(Zndx1_Jul*Zndx1_Aug)', data=test1).fit()
    if i == "IOWA":
        print(results.summary())
     
    plt.rc("figure", figsize=(20,20))
    plt.rc("font", size=14)
    
    
    from statsmodels.graphics.regressionplots import plot_leverage_resid2
    fig, ax = plt.subplots()
    fig = plot_leverage_resid2(results, ax = ax)
    
    
    fig = sm.graphics.influence_plot(results, criterion="cooks")
    fig.tight_layout(pad=1.0)
    
    
    train1 = iowa9[iowa9["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    mixed_pred =  results.predict(train1) # predict out of sample
    if i == "IOWA":
        predictions = results.get_prediction(train1)
        print(predictions.summary_frame(alpha=0.05))
    
    ##################################################
     ## Just August precip model
    test1 = iowa9[iowa9["Year"] < testYear]
    results = smf.ols('Yield ~ Year + I(Year**2) + Precip_Aug + I(Precip_Aug**2)', data=test1).fit()
    #print(results.summary())
    
    train1 = iowa9[iowa9["Year"] >= testYear]
    ynewpred =  results.predict(train1) # predict out of sample
    trend_augPrecip_pred =  results.predict(train1) # predict out of sample
    
    #######################
    output  = pd.DataFrame({"State": i,
                            "Year": range(testYear, 2021),
                            "Real*": real_pred.tolist(),
                            "trend_pred": trend_pred, 
                            "trend_sqr_pred": trend_sqr_pred,    
                            "arima_pred": arima_pred.tolist(),
                            "maxTemp_pred": maxTemp_pred.tolist(),
                            "maxT_Precip_pred": maxT_Precip_pred.tolist(),
                            "drought_pred": drought_pred.tolist(),
                            "drought_pred_zndx1": drought_pred_zndx1.tolist(),
                            "pod_pred": pod_pred.tolist(),
                            "quality_pred": quality_pred.tolist(),
                            "mixed_pred": mixed_pred.tolist(),
                            "trend_augPrecip_pred": trend_augPrecip_pred.tolist()
                            })
    myGuesses.append(output)

guesses = pd.concat(myGuesses)
guess1 = guesses.loc[:, guesses.notna().all()]
guess1.to_csv("guesses.csv", index = None)
guess1